In [29]:
import chromadb
from openapiutils import download_openapi_spec, generate_open_api_services, create_function_names_docs_import_statements, clean_url, generate_hash

chroma_client = chromadb.PersistentClient(path="agentprotocol/vectordb")


In [ ]:
doc2 = f'''
Function:
def give_joke(category : str):
"""
Joke categories. Supports: Any, Misc, Programming, Dark, Pun, Spooky, Christmas.
"""
'''

In [30]:
import subprocess
import os
import json
import yaml
from urllib.request import urlretrieve

openapi_url = "https://raw.githubusercontent.com/open-meteo/open-meteo/main/openapi.yml"
service_url = "https://api.open-meteo.com"
output_dir = generate_hash(clean_url(openapi_url))


if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    output_json_file = output_dir + "\\openapi.json"
    if not openapi_url:
        raise ValueError("Please provide a valid OpenAPI URL")

try:
    # Download the OpenAPI spec
    urlretrieve(openapi_url, output_dir + "\\openapi.yml")

    # Read the content of the file
    with open(output_dir + "\\openapi.yml", "r") as file:
        file_content = file.read()

    # Modify content (replace int and float types with number)
    file_content = file_content.replace("int\n", "number\n")
    file_content = file_content.replace("float\n", "number\n")

    # Parse the content as YAML
    data = yaml.safe_load(file_content)

    # Add servers entry (optional, modify if your OpenAPI spec requires a different structure)
    data["servers"] = [{"url": service_url}]

    # Convert to JSON and save
    with open(output_json_file, "w") as file:
        json.dump(data, file, indent=4)  # Add indentation for readability (optional)

except (ValueError, IOError) as e:
    print(f"Error: {e}")

    # Construct the command using string formatting
    command = f"openapi-python-generator {output_dir}//openapi.json {output_dir}/"  # !openapi-python-generator openapi.json ./api_specification_main/
    subprocess.run(command.split(), check=True)

    print(output_dir)

In [33]:
import subprocess

# Construct the command using string formatting
command = f"openapi-python-generator {output_dir}/openapi.json {output_dir}/"  # !openapi-python-generator openapi.json ./api_specification_main/
subprocess.run(command.split(), check=True)

openapi-python-generator a41cc9d3b6585b73a594eb1cb0cab5fcd3f67dad09b45c4572583da07b3cc751/openapi.json a41cc9d3b6585b73a594eb1cb0cab5fcd3f67dad09b45c4572583da07b3cc751/


CompletedProcess(args=['openapi-python-generator', 'a41cc9d3b6585b73a594eb1cb0cab5fcd3f67dad09b45c4572583da07b3cc751/openapi.json', 'a41cc9d3b6585b73a594eb1cb0cab5fcd3f67dad09b45c4572583da07b3cc751/'], returncode=0)

In [34]:
import os
os.remove(f"{output_dir}/openapi.yml")
os.remove(f"{output_dir}/openapi.json")

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'openapi.yml'

In [ ]:
function_names, function_docs, import_statements = create_function_names_docs_import_statements(output_dir)
print(function_docs)

In [ ]:
doc_ids = []
doc_metadatas = []
counter = 0
for function_doc in function_docs:
    doc_ids.append(str(generate_hash(function_doc)))
    doc_metadatas.append({"function_name": function_names[counter], "openapi_uri": "https://raw.githubusercontent.com/open-meteo/open-meteo/main/openapi.yml", "end_point": "https://api.open-meteo.com", "import_statement": import_statements[counter], "type": "native"})
    counter = counter + 1
    print(str(generate_hash(function_doc)))

In [ ]:
collection = chroma_client.get_or_create_collection(name="tools")
collection.add(
    documents=function_docs,
    metadatas=doc_metadatas,
    ids=doc_ids
)


In [ ]:
user_query = "Hey how is the current weather and windspeed in New York?"

In [ ]:
prompt = f'''
    Function:
    no_op():
    """
    Default OOD
    """
    '''

In [ ]:
results = collection.query(
    query_texts=[user_query], 
    n_results=2, # how many results to return
    # include=["embeddings", "metadatas", "documents", "distances"]
    include=["documents"]
)

for items in results["documents"]:
    for item in items:
        # print(str(item))
        prompt += str(item) + "\n"

prompt = prompt + f'''User Query: {user_query}<human_end>'''
print(prompt)


In [ ]:
from utils import query_raven
call = query_raven(prompt)
print (call)

In [ ]:
for import_statement in import_statements:
    exec(import_statement)
eval(call)

In [ ]:
count = len(chroma_client.list_collections())
print(count)
if count > 0:
    print(chroma_client.list_collections())

In [ ]:
chroma_client.delete_collection("tools")

In [ ]:
directory_path = 'a41cc9d3b6585b73a594eb1cb0cab5fcd3f67dad09b45c4572583da07b3cc751'
import shutil
if os.path.exists(directory_path):
    shutil.rmtree(directory_path)